# Język Python - Wykład 8

## Testowanie kodu

### assert

* w kodzie można traktować jako dodatkową dokumentację (uruchamialną)
* nie zastępują testów (jedynie uzupełniają je)
* pozwalają szybciej wyłapywać bugi

In [7]:
assert True

In [8]:
assert False, "Cos poszlo nie tak :-("

AssertionError: Cos poszlo nie tak :-(

In [9]:
assert 2+3 == 5

Niektórzy używają assertów do kontroli typów w Pythonie! (ale to jak wiemy zabija kaczki)

In [10]:
def add(a, b):
    assert type(a) is int, 'a is not an integer, a=%r' % a
    assert type(b) is int, 'b is not an integer, b=%r' % b
    return a + b

In [11]:
add(3, 5)

8

In [12]:
add(4.0, 1)

AssertionError: a is not an integer, a=4.0

Asercję można wyłączać! (np. na produkcji ze względów wydajnościowych!)

## doctest

In [13]:
def add(a, b):
    """
    Zwraca sumę dwóch liczb, np:
    
    >>> add(1, 2)
    3
    
    >>> add(1, '2')
    Traceback (most recent call last):
    ...
    TypeError: unsupported operand type(s) for +: 'int' and 'str'
    """
    return a + b 

import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

In [17]:
def factorial(n):
    """Return the factorial of n, an exact integer >= 0.

    >>> [factorial(n) for n in range(6)]
    [1, 1, 2, 6, 24, 120]
    >>> factorial(30)
    265252859812191058636308480000000
    >>> factorial(-1)
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0

    Factorials of floats are OK, but the float must be an exact integer:
    >>> factorial(30.1)
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
    >>> factorial(30.0)
    265252859812191058636308480000000

    It must also not be ridiculously large:
    >>> factorial(1e100)
    Traceback (most recent call last):
        ...
    OverflowError: n too large
    """

    import math
    if not n >= 0:
        raise ValueError("n must be >= 0")
    if math.floor(n) != n:
        raise ValueError("n must be exact integer")
    if n+1 == n:  # catch a value like 1e300
        raise OverflowError("n too large")
    result = 1
    factor = 2
    while factor <= n:
        result *= factor
        factor += 1
    return result


import doctest
doctest.testmod()

TestResults(failed=0, attempted=8)

### example.test

In [ ]:
import doctest
doctest.testfile("example.test")

In [ ]:
python -m doctest -v example.test

In [26]:
def foo(*args):
    """
    >>> print foo(range(10), range(4))
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3]
    
    >>> print foo(range(10), range(4)) # doctest: +ELLIPSIS
    [0, 1, ..., 9, 0, 1, 2, 3]
    
    >>> print foo(range(10), range(4)) # doctest: +ELLIPSIS, +NORMALIZE_WHITESPACE
    [0, ..., 9,    0, ..., 3]
    """
    return [x for el in args for x in el]

import doctest
doctest.testmod(verbose=3)

Trying:
    add(1, 2)
Expecting:
    3
ok
Trying:
    add(1, '2')
Expecting:
    Traceback (most recent call last):
    ...
    TypeError: unsupported operand type(s) for +: 'int' and 'str'
ok
Trying:
    [factorial(n) for n in range(6)]
Expecting:
    [1, 1, 2, 6, 24, 120]
ok
Trying:
    factorial(30)
Expecting:
    265252859812191058636308480000000
ok
Trying:
    factorial(-1)
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: n must be >= 0
ok
Trying:
    factorial(30.1)
Expecting:
    Traceback (most recent call last):
        ...
    ValueError: n must be exact integer
ok
Trying:
    factorial(30.0)
Expecting:
    265252859812191058636308480000000
ok
Trying:
    factorial(1e100)
Expecting:
    Traceback (most recent call last):
        ...
    OverflowError: n too large
ok
Trying:
    print foo(range(10), range(4))
Expecting:
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3]
**********************************************************************
File "__main

TestResults(failed=3, attempted=11)

## Unittest

In [30]:
import random
import unittest

class TestSequenceFunctions(unittest.TestCase):

    def setUp(self):
        self.seq = list(range(10))

    def test_shuffle(self):
        # make sure the shuffled sequence does not lose any elements
        random.shuffle(self.seq)
        self.seq.sort()
        self.assertEqual(self.seq, list(range(10)))

        # should raise an exception for an immutable sequence
        self.assertRaises(TypeError, random.shuffle, (1,2,3))

    def test_choice(self):
        element = random.choice(self.seq)
        self.assertTrue(element in self.seq)

    def test_sample(self):
        with self.assertRaises(ValueError):
            random.sample(self.seq, 20)
        for element in random.sample(self.seq, 5):
            self.assertTrue(element in self.seq)

suite = unittest.TestLoader().loadTestsFromTestCase(TestSequenceFunctions)
print(unittest.TextTestRunner(verbosity=3).run(suite))

test_choice (__main__.TestSequenceFunctions) ... ok
test_sample (__main__.TestSequenceFunctions) ... ok
test_shuffle (__main__.TestSequenceFunctions) ... 

<unittest.runner.TextTestResult run=3 errors=0 failures=0>


ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


In [34]:
import random
import unittest

class TestSequenceFunctions(unittest.TestCase):

    def setUp(self):
        self.seq = list(range(10))

    def test_shuffle(self):
        # make sure the shuffled sequence does not lose any elements
        random.shuffle(self.seq)
        self.seq.sort()
        self.assertEqual(self.seq, list(range(11)))

        # should raise an exception for an immutable sequence
        self.assertRaises(TypeError, random.shuffle, (1,2,3))

    def test_choice(self):
        element = random.choice(self.seq)
        self.assertTrue(element in self.seq)

    def test_sample(self):
        with self.assertRaises(ValueError):
            random.sample(self.seq, 20)
        for element in random.sample(self.seq, 5):
            self.assertTrue(element in self.seq)

suite = unittest.TestLoader().loadTestsFromTestCase(TestSequenceFunctions)
print(unittest.TextTestRunner(verbosity=3).run(suite))

test_choice (__main__.TestSequenceFunctions) ... ok
test_sample (__main__.TestSequenceFunctions) ... ok
test_shuffle (__main__.TestSequenceFunctions) ... 

<unittest.runner.TextTestResult run=3 errors=0 failures=1>


FAIL

FAIL: test_shuffle (__main__.TestSequenceFunctions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-34-d7e758f43733>", line 13, in test_shuffle
    self.assertEqual(self.seq, list(range(11)))
AssertionError: Lists differ: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] != [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Second list contains 1 additional elements.
First extra element 10:
10

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
+ [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
?                              ++++


----------------------------------------------------------------------
Ran 3 tests in 0.004s

FAILED (failures=1)


In [36]:
import unittest

class Widget(object):
    
    def __init__(self, name):
        self.name = name
        self._size = (50, 50)
    
    def size(self):
        return self._size
    
    def resize(self, size):
        self._size = size
        
    def dispose(self):
        pass

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def tearDown(self):
        self.widget.dispose()
        self.widget = None

    def test_default_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

suite = unittest.TestLoader().loadTestsFromTestCase(WidgetTestCase)
print(unittest.TextTestRunner(verbosity=3).run(suite))

test_default_size (__main__.WidgetTestCase) ... ok
test_resize (__main__.WidgetTestCase) ... 

<unittest.runner.TextTestResult run=2 errors=1 failures=0>


ERROR

ERROR: test_resize (__main__.WidgetTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-36-c635f2067186>", line 31, in test_resize
    self.widget.resize(100,150)
TypeError: resize() takes 2 positional arguments but 3 were given

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (errors=1)


In [38]:
import unittest

class Widget(object):
    
    def __init__(self, name):
        self.name = name
        self._size = (50, 50)
    
    def size(self):
        return self._size
    
    def resize(self, width, height):
        self._size = (width, height)
        
    def dispose(self):
        pass

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def tearDown(self):
        self.widget.dispose()
        self.widget = None

    def test_default_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

suite = unittest.TestLoader().loadTestsFromTestCase(WidgetTestCase)
print(unittest.TextTestRunner(verbosity=3).run(suite))

test_default_size (__main__.WidgetTestCase) ... ok
test_resize (__main__.WidgetTestCase) ... 

<unittest.runner.TextTestResult run=2 errors=0 failures=0>


ok

----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK
